In [ ]:
import heapq

# --- Graph Representation from the first image ---
# The graph is represented as an adjacency list:
# {node: [(neighbor, weight), ...]}
graph = {
    1: [(3, 5), (5, 2)],
    2: [(4, 7), (5, 1), (6, 8)],
    3: [(1, 5), (6, 5)],
    4: [(2, 7), (5, 1)],
    5: [(1, 2), (2, 1), (4, 1), (6, 3)],
    6: [(2, 8), (3, 5), (5, 3)]
}

# --- Dijkstra's Algorithm Implementation ---
def dijkstra(graph, start_node):
    """
    Finds the shortest paths from a start_node to all other nodes
    in a graph with non-negative edge weights using Dijkstra's algorithm.

    Args:
        graph (dict): Adjacency list representation of the graph.
                      {node: [(neighbor, weight), ...]}
        start_node: The starting node for finding shortest paths.

    Returns:
        dict: A dictionary where keys are nodes and values are their
              shortest distances from the start_node.
              Returns float('inf') for unreachable nodes.
    """
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0
    priority_queue = [(0, start_node)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # If we've already found a shorter path to this node, skip
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight
            # If a shorter path to the neighbor is found
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

print("--- Dijkstra's Algorithm Execution Example (from Node 1) ---")
start_node_for_dijkstra = 1
shortest_distances_dijkstra = dijkstra(graph, start_node_for_dijkstra)
print(f"Shortest distances from node {start_node_for_dijkstra}:")
for node, dist in shortest_distances_dijkstra.items():
    print(f"  To node {node}: {dist}")

print("\n" + "="*50 + "\n")

# --- Bellman-Ford Algorithm Implementation ---
def bellman_ford(graph, start_node):
    """
    Finds the shortest paths from a start_node to all other nodes
    in a graph (can have negative edge weights) using Bellman-Ford.
    Also detects negative cycles.

    Args:
        graph (dict): Adjacency list representation of the graph.
                      {node: [(neighbor, weight), ...]}
        start_node: The starting node for finding shortest paths.

    Returns:
        tuple: A tuple containing:
            - dict: Shortest distances from start_node.
            - bool: True if a negative cycle is detected, False otherwise.
    """
    nodes = list(graph.keys())
    distances = {node: float('inf') for node in nodes}
    distances[start_node] = 0

    # Relax all edges |V| - 1 times
    for _ in range(len(nodes) - 1):
        for u in nodes:
            for v, weight in graph.get(u, []): # Use .get(u, []) to handle nodes not in graph's keys explicitly
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

    # Check for negative cycles
    has_negative_cycle = False
    for u in nodes:
        for v, weight in graph.get(u, []):
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                has_negative_cycle = True
                break
        if has_negative_cycle:
            break

    return distances, has_negative_cycle

print("--- Bellman-Ford Algorithm Implementation (Theoretical, no negative weights in current graph) ---")
# Example usage (using the same graph, but remember Bellman-Ford handles negative weights)
start_node_for_bellman_ford = 1
bf_distances, bf_has_negative_cycle = bellman_ford(graph, start_node_for_bellman_ford)
print(f"Shortest distances from node {start_node_for_bellman_ford} (Bellman-Ford):")
for node, dist in bf_distances.items():
    print(f"  To node {node}: {dist}")
if bf_has_negative_cycle:
    print("Negative cycle detected!")
else:
    print("No negative cycle detected.")

print("\n" + "="*50 + "\n")

# --- Floyd-Warshall Algorithm Implementation ---
def floyd_warshall(graph):
    """
    Finds the shortest paths between all pairs of nodes in a graph
    using the Floyd-Warshall algorithm. Handles negative edge weights,
    but not negative cycles that affect reachability.

    Args:
        graph (dict): Adjacency list representation of the graph.
                      {node: [(neighbor, weight), ...]}

    Returns:
        tuple: A tuple containing:
            - dict: A dictionary of dictionaries, where result[i][j] is
                    the shortest distance from node i to node j.
            - bool: True if a negative cycle is detected (diagonal becomes negative), False otherwise.
    """
    nodes = sorted(list(graph.keys()))
    num_nodes = len(nodes)
    node_to_idx = {node: i for i, node in enumerate(nodes)}
    idx_to_node = {i: node for i, node in enumerate(nodes)}

    # Initialize distance matrix
    dist = [[float('inf')] * num_nodes for _ in range(num_nodes)]

    # Set distance to self as 0
    for i in range(num_nodes):
        dist[i][i] = 0

    # Populate initial distances from direct edges
    for u_node in graph:
        for v_node, weight in graph[u_node]:
            u_idx = node_to_idx[u_node]
            v_idx = node_to_idx[v_node]
            dist[u_idx][v_idx] = min(dist[u_idx][v_idx], weight) # Handle parallel edges, although not in given graph

    # Apply Floyd-Warshall
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if dist[i][k] != float('inf') and dist[k][j] != float('inf'):
                    dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])

    # Check for negative cycles (if any diagonal entry is negative)
    has_negative_cycle = False
    for i in range(num_nodes):
        if dist[i][i] < 0:
            has_negative_cycle = True
            break

    # Convert back to node labels for output
    all_pairs_shortest_paths = {}
    for i in range(num_nodes):
        all_pairs_shortest_paths[idx_to_node[i]] = {}
        for j in range(num_nodes):
            all_pairs_shortest_paths[idx_to_node[i]][idx_to_node[j]] = dist[i][j]

    return all_pairs_shortest_paths, has_negative_cycle

print("--- Floyd-Warshall Algorithm Implementation ---")
fw_distances, fw_has_negative_cycle = floyd_warshall(graph)
print("All-Pairs Shortest Paths (Floyd-Warshall):")
for source_node, targets in fw_distances.items():
    print(f"From node {source_node}:")
    for target_node, dist in targets.items():
        print(f"  To node {target_node}: {dist}")
if fw_has_negative_cycle:
    print("Negative cycle detected!")
else:
    print("No negative cycle detected.")


print("\n" + "="*50 + "\n")
print("--- ВІДПОВІДІ НА КОНТРОЛЬНІ ЗАПИТАННЯ ---")

# Question 1
print("\n1. Що таке граф і які головні складові його структури?")
print("""
Граф - це абстрактна математична структура, що використовується для моделювання зв'язків між об'єктами. Він складається з двох основних множин:
* Вершини (Nodes/Vertices): Це об'єкти, що представляються точками.
* Ребра (Edges/Links): Це зв'язки або відношення між парами вершин. Ребра можуть бути:
    * Неорієнтовані: Зв'язок є двонаправленим (наприклад, дружба у соціальній мережі).
    * Орієнтовані (Digraphs): Зв'язок є односпрямованим (наприклад, вулиця з одностороннім рухом).
    * Зважені: Кожне ребро має асоційоване числове значення (вагу або вартість), яке може представляти відстань, час, ціну тощо.
""")

# Question 2
print("\n2. Які алгоритми використовуються для пошуку найкоротших шляхів у графах?")
print("""
Для пошуку найкоротших шляхів у графах використовуються різні алгоритми, вибір яких залежить від властивостей графа (наявність від'ємних ваг, орієнтований/неорієнтований) та кількості вихідних/цільових вершин:
* Алгоритм Дейкстри (Dijkstra's Algorithm): Знаходить найкоротші шляхи від однієї початкової вершини до всіх інших вершин у графах без від'ємних ваг ребер.
* Алгоритм Беллмана-Форда (Bellman-Ford Algorithm): Знаходить найкоротші шляхи від однієї початкової вершини до всіх інших вершин у графах, які можуть містити від'ємні ваги ребер. Також може виявляти від'ємні цикли.
* Алгоритм Флойда-Воршелла (Floyd-Warshall Algorithm): Знаходить найкоротші шляхи між усіма парами вершин у графі. Працює з від'ємними вагами, але не з від'ємними циклами.
* Алгоритм А* (A* Search Algorithm): Це розширення алгоритму Дейкстри, що використовує "евристику" для ефективнішого пошуку найкоротшого шляху між двома конкретними вершинами. Часто використовується в задачах пошуку шляху в іграх та навігації.
* BFS (Breadth-First Search - Пошук у ширину): Якщо всі ваги ребер дорівнюють 1 (незважені графи), BFS ефективно знаходить найкоротший шлях (у термінах кількості ребер) від однієї вершини до всіх досяжних вершин.
""")

# Question 3
print("\n3. Як працює алгоритм Дейкстри і які його особливості?")
print("""
Принцип роботи:
Алгоритм Дейкстри знаходить найкоротші шляхи від однієї початкової вершини (джерела) до всіх інших вершин у графі з невід'ємними вагами ребер. Він працює за принципом "жадібного" підходу, поступово розширюючи множину "відомих" вершин, до яких вже знайдено найкоротший шлях.

1.  Ініціалізація:
    * Всім вершинам призначається "відстань" (довжина найкоротшого шляху від джерела) нескінченність, крім початкової вершини, якій присвоюється 0.
    * Створюється множина відвіданих вершин (або пріоритетна черга).
2.  Ітерації:
    * На кожному кроці алгоритм вибирає невідвідану вершину з найменшою поточною відстанню.
    * Ця вершина позначається як відвідана.
    * Для всіх сусідів обраної вершини перевіряється, чи можна знайти коротший шлях до них через поточну вершину. Якщо так, відстань до сусіда оновлюється.
3.  Завершення: Алгоритм продовжується, доки всі вершини не будуть відвідані або поки всі досяжні вершини не будуть опрацьовані.

Особливості:
* Придатність: Працює лише для графів без від'ємних ваг ребер. Наявність від'ємних ваг може призвести до неправильних результатів, оскільки алгоритм не перевіряє вже "завершені" вершини на можливі подальші скорочення шляху.
* Оптимальність: Гарантує знаходження найкоротших шляхів, якщо умови (невід'ємні ваги) дотримані.
* Складність: Залежить від структури даних для пріоритетної черги. З використанням бінарної купи складність складає $O(E \log V)$ або $O((V+E) \log V)$, де $V$ - кількість вершин, $E$ - кількість ребер. З використанням фібоначчієвої купи - $O(E + V \log V)$.
* Вихід: Результатом є масив (або словник) з найкоротшими відстанями від джерела до кожної іншої вершини.
""")

# Question 4
print("\n4. Що таке алгоритм Беллмана-Форда і коли його варто застосовувати?")
print("""
Що це:
Алгоритм Беллмана-Форда - це алгоритм пошуку найкоротших шляхів від однієї початкової вершини до всіх інших вершин у зваженому графі. На відміну від алгоритму Дейкстри, він може обробляти графи, що містять ребра з від'ємними вагами. Він також може виявити наявність від'ємних циклів, які роблять поняття найкоротшого шляху невизначеним (можна постійно зменшувати шлях, проходячи через від'ємний цикл).

Принцип роботи:
Алгоритм працює шляхом багаторазового "послаблення" (relaxation) всіх ребер у графі. "Послаблення" полягає у перевірці, чи можна знайти коротший шлях до кінцевої вершини ребра, проходячи через його початкову вершину.

1.  Ініціалізація: Встановлюється відстань до початкової вершини як 0, а до всіх інших вершин як нескінченність.
2.  Ітерації (V-1 разів): Алгоритм повторює прохід по всіх ребрах графа V-1 разів (де V - кількість вершин). На кожній ітерації для кожного ребра (u, v) з вагою w:
    * Якщо dist[u] + w < dist[v], то dist[v] оновлюється до dist[u] + w.
    * Цей процес гарантує, що після V-1 ітерації всі найкоротші шляхи (якщо немає від'ємних циклів) будуть знайдені.
3.  Виявлення від'ємних циклів: Після V-1 ітерації виконується ще одна (V-та) ітерація. Якщо на цій ітерації відбувається будь-яке подальше оновлення відстаней, це означає, що в графі існує досяжний від'ємний цикл, і найкоротші шляхи не можуть бути визначені.

Коли застосовувати:
* Графи з від'ємними вагами ребер: Це основний сценарій, коли Беллман-Форд є кращим за Дейкстру.
* Виявлення від'ємних циклів: Якщо необхідно не лише знайти найкоротші шляхи, але й перевірити наявність від'ємних циклів, Беллман-Форд є підходящим.
* Single-Source Shortest Path (одне джерело): Для пошуку шляхів від однієї вершини до всіх інших.

Недоліки:
* Складність: $O(V \cdot E)$, що значно повільніше, ніж Дейкстра для графів без від'ємних ваг. Тому для графів без від'ємних ваг Дейкстра є більш ефективним.
""")

# Question 5
print("\n5. Як працює алгоритм Флойда-Воршелла і які його переваги та недоліки?")
print("""
Принцип роботи:
Алгоритм Флойда-Воршелла - це алгоритм динамічного програмування, який знаходить найкоротші шляхи між усіма парами вершин у зваженому орієнтованому або неорієнтованому графі. Він може працювати з графами, що містять від'ємні ваги ребер, але не може працювати, якщо в графі є від'ємні цикли.

Алгоритм будує матрицю відстаней $dist[i][j]$, яка на початку ініціалізується прямими вагами ребер (якщо ребро (i,j) існує) або нескінченністю (якщо ребро відсутнє). Діагональні елементи $dist[i][i]$ дорівнюють 0.

Його основна ідея полягає в ітеративному поліпшенні оцінок найкоротших шляхів, розглядаючи кожну вершину $k$ як можливу "проміжну" вершину на шляху між будь-якою парою вершин (i, j).
for k from 1 to V:
for i from 1 to V:
for j from 1 to V:
dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
Тобто, для кожної пари вершин (i, j), алгоритм перевіряє, чи шлях $i \to k \to j$ коротший, ніж поточний найкоротший шлях $i \to j$.

Переваги:
* All-Pairs Shortest Path: Знаходить найкоротші шляхи між усіма парами вершин одночасно. Це зручно, якщо потрібно знати всі можливі найкоротші шляхи в графі.
* Простота реалізації: Має відносно простий і лаконічний код (три вкладені цикли).
* Підтримка від'ємних ваг: Може працювати з ребрами, що мають від'ємні ваги.
* Виявлення від'ємних циклів: Якщо після виконання алгоритму $dist[i][i]$ стає від'ємним для будь-якої вершини $i$, це свідчить про наявність від'ємного циклу, досяжного з $i$.

Недоліки:
* Висока обчислювальна складність: Має складність $O(V^3)$, що робить його менш ефективним для дуже великих графів порівняно з послідовним запуском Дейкстри з кожної вершини (якщо немає від'ємних ваг).
* Не підходить для розріджених графів: Для графів з великою кількістю вершин, але невеликою кількістю ребер (розріджені графи), $O(V \cdot E \log V)$ або $O(V \cdot (E + V \log V))$ (запуск Дейкстри V разів) може бути швидшим, ніж $O(V^3)$ Флойда-Воршелла.
* Не виводить шлях: Зазвичай алгоритм повертає лише матрицю відстаней. Щоб відновити самі шляхи, потрібно зберігати додаткову матрицю попередників.
""")
